pyTorch Wordle

In [145]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Generate Dataset

In [146]:
class wordleDataset(Dataset):
    def __init__(self,data_dir,targetToClass,transform=None):
        self.data = []
        with open(data_dir,'r') as f:
            for line in f:
                lineData = line.split()
                target = lineData[0]
                guesses = lineData[1:len(lineData)]

                #Tokenize the inputs and convert to tensors
                
                target = targetToClass[target]
                for i in range(len(guesses)):
                    guesses[i] = targetToClass[guesses[i]]
                targetTensor = torch.tensor(target)
                guessTensor = torch.tensor(guesses)
                self.data.append([targetTensor,guessTensor])
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [147]:
wordToNum = {}
numToWord = {}
index = 0
with open("words.txt",'r') as f:
    for line in f:
        lineData = line.split()
        numToWord.update({index:lineData[0]})
        wordToNum.update({lineData[0]:index})
        index += 1

wordleSet = wordleDataset("TrainingData.txt",wordToNum)


In [148]:
dataloader = DataLoader(wordleSet, batch_size=32, shuffle=True)
for outputData,inputData in dataloader:
    break

Training

In [149]:
class neuralNetwork(nn.Module):
    def __init__(self,num_in,h1,h2,num_out):
        super().__init__()
        self.fc1 = nn.Linear(num_in,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2,num_out)


    def forward(self,x):
        #Inputs into fc1 through relu function (f(x) = x if x>0,else 0)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.out(x)
        
        return x

In [150]:
torch.manual_seed(234524)

model = neuralNetwork(10,256,256,15099)

In [151]:
critereon = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
epochNum = 100
losses,valLosses = [],[]
for i in range(epochNum):
    print(i,epochNum)
    
    model.train()
    curLoss = 0
    index=0
    for output,input in dataloader:
        index+=1
        optimizer.zero_grad()
        input = input.float()

        modelOutputs = model(input)
        loss = critereon(modelOutputs,output)
        loss.backward()
        optimizer.step()
        curLoss += loss.item()*len(input)
    totalLoss = curLoss / len(dataloader.dataset)
    losses.append(totalLoss)

    #Evaluation
    model.eval()
    curLoss=0
    with torch.no_grad():
        for output,input in dataloader:
            input = input.float()

            modelOutputs = model(input)
            loss = critereon(modelOutputs,output)
            curLoss += loss.item() * len(input)
    valLoss = curLoss / len(dataloader.dataset)
    valLosses.append(curLoss)

    print(totalLoss)


0 100
35.656976654052734
1 100
9.544639926147461
2 100
9.44356936126709
3 100
9.395314634399414
4 100
9.370790156860352
5 100
9.357799973449707
6 100
9.351068764648437
7 100
9.34757723388672
8 100
9.345680192565919
9 100
9.344569777526855
10 100
9.344097963562012
11 100
9.343771035461426
12 100
9.343572631225586
13 100
9.343468377990723
14 100
9.34359139465332
15 100
9.34335033782959
16 100
9.34339220336914
17 100
9.34335113067627
18 100
9.343204164123534
19 100
9.34341103149414
20 100
9.343266569519043
21 100
9.343094058532715
22 100
9.343030309448242
23 100
9.343252382202149
24 100
9.343185375976562
25 100
9.343266513671875
26 100
9.343257315063477
27 100
9.343205603637696
28 100
9.343257147216796
29 100
9.34313479309082
30 100
